In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 1.15

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [4]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [5]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
#from attention import AttentionLayer
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [6]:
# Libary
import os
import re
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt

In [7]:
dataset_dir = '/content/drive/MyDrive/PROJECT/down_total_data/dataset'
dataset_txt = os.listdir(dataset_dir)
dataset_txt.sort()
# dataset_txt[0:5]
print(dataset_txt)

['003. putusan_851_k_pdt.sus-phi_2020_20210305.txt', '006. putusan_1112_k_pdt.sus-phi_2020_20210305.txt', '007. putusan_1182_k_pdt.sus-phi_2020_20210305.txt', '008. putusan_1195_k_pdt.sus-phi_2020_20210305.txt', '009. putusan_1197_k_pdt.sus-phi_2020_20210304.txt', '010. putusan_1197_k_pdt.sus-phi_2020_20210305.txt', '011. putusan_1256_k_pdt.sus-phi_2020_20210305.txt', '012. putusan_1258_k_pdt.sus-phi_2020_20210304.txt', '013. putusan_1258_k_pdt.sus-phi_2020_20210305.txt', '015. putusan_1277_k_pdt.sus-phi_2020_20210305.txt', '018. putusan_1312_k_pdt.sus-phi_2020_20210305.txt', '020. putusan_1321_k_pdt.sus-phi_2020_20210304.txt', '022. putusan_1352_k_pdt.sus-phi_2020_20210304.txt', '023. putusan_1354_k_pdt.sus-phi_2020_20210304.txt', '024. putusan_1354_k_pdt.sus-phi_2020_20210305.txt', '032. putusan_1429_k_pdt.sus-phi_2020_20210304.txt', '033. putusan_1431_k_pdt.sus-phi_2020_20210304.txt', '034. putusan_1431_k_pdt.sus-phi_2020_20210305.txt', '035. putusan_7_k_pdt.sus-phi_2020_20201117.tx

In [8]:
# fungsi load data txt
def load_data(path_doc):
  path_doc = open(path, "r")
  documents_list = []
  for line in path_doc.readlines():
    # lower = line.lower()
    text = line.strip()
    documents_list.append(text)
  return documents_list

# case folding
def case_folding(list_dokumen):
  doc_case = []
  for line in list_dokumen:
    lower = line.lower()
    doc_case.append(lower)
  return doc_case

# hapus whitespace
def whitespace_remove(list_dokumen):
  while("" in list_dokumen) : 
    list_dokumen.remove("") 
  return list_dokumen 

# hapus header footer
def remove_hf(list_dokumen):
  HF = [
    'Disclaimer',
    'Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu. Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :',
    'Direktori Putusan Mahkamah Agung Republik Indonesia',
    'putusan.mahkamahagung.go.id'
  ]
  count = 0
  for i in list_dokumen: 
    if i == HF[0].lower():
      list_dokumen[count-1] = ""
    if i == HF[1].lower():
      list_dokumen[count+1] = ""
    if i == HF[0].lower() or i == HF[1].lower() or i == HF[2].lower() or i == HF[3].lower():
      list_dokumen[count] = ""
    count = count+1
  whitespace_remove(list_dokumen)
  return list_dokumen

# hapus numbering
def numbering_remove(list_dokumen,numbering):
  for i in range(len(list_dokumen)):
    if list_dokumen[i] == numbering:
      list_dokumen[i] = ""
  return list_dokumen


In [9]:
doc_clean_all = []
for i in dataset_txt:
  # load data
  path = "/content/drive/MyDrive/PROJECT/down_total_data/dataset/"+str(i)
  doc = load_data(path)

  # case folding
  cf = case_folding(doc)
  
  # white space
  wr = whitespace_remove(cf)

  # header & footer & page number
  hf = remove_hf(wr)
  
  # masukkan ke list
  doc_clean_all.append(hf)
  
# menampilkan isi semua dokumen pada list
doc_clean_all[2]

['p u t u s a n',
 'nomor 1182 k/pdt.sus-phi/2020',
 'demi keadilan berdasarkan ketuhanan yang maha esa m a h k a m a h   a g u n g',
 'memeriksa perkara perdata khusus perselisihan hubungan industrial pada tingkat kasasi memutus sebagai berikut dalam perkara antara:',
 'heara halawa, bertempat tinggal di kampung leuweung',
 'kaleng, rt 04, rw 03, kecamatan katapang, kabupaten',
 'bandung; pemohon kasasi;',
 'l a w a n',
 'suryanto, hrd pt security phisik dinamika, berkedudukan di jalan bina marga, nomor 30, kelurahan sanja, kecamatan citeureup, bogor;',
 'termohon kasasi; mahkamah agung tersebut;',
 'membaca surat-surat yang bersangkutan yang merupakan bagian tidak terpisahkan dari putusan ini;',
 'menimbang, bahwa berdasarkan surat-surat yang bersangkutan, penggugat telah mengajukan gugatan di depan persidangan pengadilan hubungan industrial pada pengadilan negeri bandung dan memohon kepada pengadilan untuk memberikan putusan sebagai berikut:',
 '1.   menyatakan menerima dan mengabul

In [10]:
#mencari index kalimat berdasarkan kata cari 5 inputan
def search_index_expert(list_dokumen,cari_string1,cari_string2,cari_string3,cari_string4,cari_string5,cari_string6):
  index_kata = 0
  for line in list_dokumen:
    if cari_string1 in line: 
      break
    elif cari_string2 in line: 
      break
    elif cari_string3 in line: 
      break
    elif cari_string4 in line: 
      break
    elif cari_string5 in line: 
      break
    elif cari_string6 in line: 
      break
    index_kata = index_kata + 1
  return index_kata

def get_part_doc2(list_dokumen,start,end):
  index_awal = search_index_expert(list_dokumen,start[0],start[1],start[2],start[3],start[4],start[5]) + 1;
  index_akhir = search_index_expert(list_dokumen,end[0],end[1],end[2],end[3],end[4],end[5]) + 1;
  doc = list_dokumen[index_awal : index_akhir]
  return doc

In [11]:
kata_kunci_awal = [
    "dalam pokok perkara:",
    "dalam pokok perkara :",
    "primair:",
    "subdair:",
    "menimbang,",
    "menimbang, ",
]
kata_kunci_akhir = [
    "keberatan-keberatan ",
    "keberatan-keberatan",
    "alasan-alasan",
    "alasan-alasan ",
    "pertimbangan",
    "pertimbangan ",
  #'tentang pertimbangan hukum', 
]

final_text = []
for i in doc_clean_all:
  final_text_dirty = get_part_doc2(i,kata_kunci_awal,kata_kunci_akhir)
  # cleaning pokok perkara
  data = [] 
  for i in final_text_dirty:
    print(i)
    split = i.split(" ")

    # hapus bullet numbering
    for i in range(1,100):
      numbering = "{}.".format(i)
      rm_num = numbering_remove(split,numbering) 
    rm_bul = numbering_remove(rm_num,"-") 

    # hapus whitespace
    clean = whitespace_remove(rm_bul)
    text = " ".join(clean)

    # masukkan kedalam list
    data.append(text)
  final_text.append(data)

#final_text[0]

penggugat telah  mengajukan gugatan  di  depan  persidangan pengadilan
hubungan industrial pada pengadilan negeri ternate dan memohon kepada
pengadilan untuk memberikan putusan sebagai berikut: dalam provisi:
1.   mengabulkan permohonan provisi dari penggugat untuk seluruhnya;
2.   menghukum  tergugat  untuk  membayar  terlebih  dahulu  (uang  muka) biaya transportasi dan akomodasi penggugat dalam proses pengajuan gugatan pemutusan hubungan kerja di pengadilan hubungan industrial pada pengadilan negeri ternate sebesar rp9.000.000,00 x 12 persidangan = rp108.000.000,00 (seratus delapan juta rupiah). setelah putusan ini dibacakan dalam persidangan;
dalam pokok perkara:
1.   menerima gugatan penggugat untuk seluruhnya;
2.   menyatakan   pemutusan   hubungan   kerja   yang   dilakukan   tergugat terhadap penggugat pada tanggal 24 desember 2018 batal demi hukum;
3.  menyatakan tergugat telah melakukan perbuatan yang bertentangan dengan undang undang nomor 13 tahun 2003 tentang ketenagakerja

In [12]:
# fungsi menghapus karakter tidak penting pada list
def hapus_karakter_tidak_penting(dokumen_list,karakter_dihapus):
  while(karakter_dihapus in dokumen_list) : 
    dokumen_list.remove(karakter_dihapus) 
  return dokumen_list

In [13]:
# pembobotan matriks
def pembobotan_matriks(dokumen_putusan_list):
  from sklearn.feature_extraction.text import CountVectorizer
  vectorizer = CountVectorizer()
  bag_of_words = vectorizer.fit_transform(dokumen_putusan_list)
  bag_of_words.todense()
  return bag_of_words

In [14]:
# dekomposisi matriks
def dekomposisi_matriks(hasil_pembobotan_matriks):
  from sklearn.decomposition import TruncatedSVD
  svd = TruncatedSVD(n_components = 1)
  decomposition = svd.fit_transform(hasil_pembobotan_matriks)
  return decomposition

In [15]:
# normalisasi data
def normalisasi_data_dan_input_ke_table(dokumen_putusan_clean,hasil_dekomposisi):
  import pandas as pd
  topic_encoded_df = pd.DataFrame(hasil_dekomposisi, columns=["bobot_dokumen"])
  topic_encoded_df["dokumen_clean"]= dokumen_putusan_clean
  topic_encoded_df["bobot_dokumen"] = topic_encoded_df["bobot_dokumen"] / topic_encoded_df["bobot_dokumen"].max()
  return topic_encoded_df

# normalisasi data
def normalisasi_data_dan_input_ke_table2(hasil_dekomposisi):
  import pandas as pd
  topic_encoded_df = pd.DataFrame(hasil_dekomposisi, columns=["bobot_dokumen"])
  topic_encoded_df["bobot_dokumen"] = topic_encoded_df["bobot_dokumen"] / topic_encoded_df["bobot_dokumen"].max()
  return topic_encoded_df

In [16]:
# batasi angka desimal di pandas
def batasi_desimal_pandas(data_frame_pandas,nama_colom):
  for i in range(0,len(data_frame_pandas)):
    bobot = "{:.4f}".format(data_frame_pandas.loc[i,nama_colom])
    data_frame_pandas.loc[i,nama_colom] = float(bobot)
  return data_frame_pandas

In [17]:
all_bobot = []
for i in range(0,len(dataset_txt)):
  # pembobotan & input matriks
  hasil_pembobotan_matriks = pembobotan_matriks(final_text[i])
  all_bobot.append(hasil_pembobotan_matriks)

# print(all_bobot[i].todense())

## lihat semua
for i in range(0,5):
  print(all_bobot[i].todense())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [1 1 0 ... 1 0 2]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]


In [18]:
dekomposisi_dict = {}
for i in range(0,len(dataset_txt)):
  # dekomposisi matriks
  hasil_dekomposisi = dekomposisi_matriks(all_bobot[i])

  # normalisasi data
  endcoder_data = normalisasi_data_dan_input_ke_table2(hasil_dekomposisi)
  endcoder_data = batasi_desimal_pandas(endcoder_data,'bobot_dokumen')

  normalisasi = []
  for k in endcoder_data['bobot_dokumen']:
    normalisasi.append(k)

  # pandas to dict
  name = "doc_{}".format(i+1)
  dekomposisi_dict[name] = normalisasi

# lihat semua
for j in range(5):
  name = "doc_{}".format(j+1)
  print("{} : {}".format(name,dekomposisi_dict[name]))

doc_1 : [0.1053, 0.2216, 0.1152, 0.0703, 0.8789, 0.0154, 0.052, 0.228, 0.1266, 0.3034, 0.9023, 0.6966, 0.1162, 0.0574, 0.6226, 0.3502, 0.0985, 0.2247, 0.0732, 0.5186, 0.0121, 0.0481, 0.0478, 0.1653, 0.6016, 0.3377, 0.0676, 0.5415, 0.1473, 1.0, 0.2696, 0.3069, 0.1194, 0.4275, 0.0416, 0.0735, 0.1222, 0.0749, 0.0217, 0.0571, 0.1222, 0.0693, 0.0175, 0.3712, 0.0823, 0.0296, 0.0285, 0.1057, 0.0518, -0.0, 0.3669, 0.0406]
doc_2 : [0.0678, 0.2325, 0.0057, 0.0167, 0.1687, 0.1434, 0.0862, 0.2506, 0.0705, 0.0186, 0.0043, 0.0043, 0.0045, 0.0001, 0.114, 0.0112, 0.0697, 0.0047, 0.086, 0.0594, 0.1045, 0.0, 0.4572, 0.0166, 0.1222, 0.1223, 0.076, 0.1061, 0.0052, 0.0053, 0.0001, 0.0408, 0.0017, 0.6653, 1.0, 0.2872, 0.3167, 0.0984, 0.1655, 0.1506, 0.0386, 0.0078, 0.0186, 0.3774, 0.0524]
doc_3 : [0.0703, 0.488, 0.0345, 0.0096, 0.1005, 0.0653, 0.3603, 0.1023, 0.2239, 0.6341, 0.0194, 0.5609, 0.0587, 0.1693, 0.2272, 0.7857, 0.0801, 0.1627, 0.0551, 0.0656, 0.0203, 0.0203, 0.0213, 0.0203, 0.0586, 0.0, 0.0409, 0

In [19]:
# hapus penomoran baik angka, nomor dari hasil rangkuman
def hapus_bullets_and_numbering(dokumen_terpilih_lsa):
  hasil_rangkum_lsa = dokumen_terpilih_lsa
  count = 0
  for i in hasil_rangkum_lsa:
    index = 0
    for text in i:
        if text == "a." or text == "b." or text == "c." or text == "d." or text == "e." or text == "f." or text == "g.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "h." or text == "i." or text == "j." or text == "k." or text == "l." or text == "m." or text == "n.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "o." or text == "p." or text == "q." or text == "r." or text == "s." or text == "t." or text == "u.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "v." or text == "w." or text == "x." or text == "y." or text == "z.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "1." or text == "2." or text == "3." or text == "4." or text == "5." or text == "6." or text == "7.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "8." or text == "9." or text == "10." or text == "11." or text == "12." or text == "13.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "14." or text == "15." or text == "16." or text == "17." or text == "18." or text == "19." or text == "20.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "21." or text == "22." or text == "23." or text == "24." or text == "25." or text == "26." or text == "27.":
          hasil_rangkum_lsa[count][index] = ""
        elif text == "- " or text == "-" or text == " - ":
          hasil_rangkum_lsa[count][index] = ""
        else:
          hasil_rangkum_lsa[count][index] = text
        index +=1
    count += 1
    for i in hasil_rangkum_lsa:
      hapus_karakter_tidak_penting(i,"")
    for i in range(len(hasil_rangkum_lsa)):
      hasil_rangkum_lsa[i] = " ".join(hasil_rangkum_lsa[i])
    return hasil_rangkum_lsa

In [20]:
# fungsi LSA
def LSA(dokumen_hasil_preprocessing,compression_rate):
  # pembobotan matriks
  hasil_pembobotan_matriks = pembobotan_matriks(dokumen_hasil_preprocessing)

  # dekomposisi matriks
  hasil_dekomposisi = dekomposisi_matriks(hasil_pembobotan_matriks)

  # encoder data
  endcoder_data = normalisasi_data_dan_input_ke_table(dokumen_hasil_preprocessing,hasil_dekomposisi)
  endcoder_data = batasi_desimal_pandas(endcoder_data,'bobot_dokumen')

  # ekstraksi hasil rangkuman_lsa
  compression_rate = compression_rate #40%
  rangkuman_lsa=endcoder_data[endcoder_data["bobot_dokumen"]>=compression_rate]
  rangkuman_lsa[['dokumen_clean','bobot_dokumen']]

  hasil_lsa = []
  for i in rangkuman_lsa['dokumen_clean']:
    hasil_lsa.append(i.split())

  # join kalimat
  hasil_lsa = hapus_bullets_and_numbering(hasil_lsa)
  hasil_lsa = ". ".join(hasil_lsa)
  return hasil_lsa

In [21]:
# fungsi CLSA
def CLSA(dokumen_hasil_preprocessing):
  # pembobotan matriks
  hasil_pembobotan_matriks = pembobotan_matriks(dokumen_hasil_preprocessing)

  # dekomposisi matriks
  hasil_dekomposisi = dekomposisi_matriks(hasil_pembobotan_matriks)

  # encoder data
  endcoder_data = normalisasi_data_dan_input_ke_table(dokumen_hasil_preprocessing,hasil_dekomposisi)
  endcoder_data = batasi_desimal_pandas(endcoder_data,'bobot_dokumen')

  # ekstraksi hasil rangkuman_clsa
  mean = endcoder_data["bobot_dokumen"].mean()
  compression_rate = mean
  rangkuman_clsa=endcoder_data[endcoder_data["bobot_dokumen"]>=compression_rate]
  rangkuman_clsa[['dokumen_clean','bobot_dokumen']]

  hasil_clsa = []
  for i in rangkuman_clsa['dokumen_clean']:
    hasil_clsa.append(i.split())

  # join kalimat
  hasil_clsa = hapus_bullets_and_numbering(hasil_clsa)
  hasil_clsa = ". ".join(hasil_clsa)
  return hasil_clsa

In [22]:
all_lsa_pokok_perkara = []
compression_rate = 0.4
for i in range(0,len(dataset_txt)):
  all_lsa_pokok_perkara.append(LSA(final_text[i],compression_rate))

all_lsa_pokok_perkara[0:5]

['menghukum tergugat untuk membayar terlebih dahulu (uang muka) biaya transportasi dan akomodasi penggugat dalam proses pengajuan gugatan pemutusan hubungan kerja di pengadilan hubungan industrial pada pengadilan negeri ternate sebesar rp9.000.000,00 x 12 persidangan = rp108.000.000,00 (seratus delapan juta rupiah). setelah putusan ini dibacakan dalam persidangan;. menghukum tergugat untuk membayar pesangon dan kompensasi sebesar 3 x pkb yakni 3 x 13,42 bulan x rp64.464.000,00 (enam puluh empat juta empat ratus enam puluh empat ribu rupiah) = rp2.595.320.640,00 (dua miliar lima ratus sembilan puluh lima juta tiga ratus dua puluh ribu enam ratus empat puluh rupiah);. menghukum tergugat untuk membayar upah dan tunjangan penggugat selama proses penyelesaian yakni tanggal 15 januari 2019 sampai ada putusan pengadilan yang berkekuatan hukum tetap sebesar rp74.800.266,00 (tujuh puluh empat juta delapan ratus ribu dua ratus enam puluh enam rupiah) untuk setiap bulannya;. hubungan kerja di pen

In [23]:
all_clsa_pokok_perkara = []
for i in range(0,len(dataset_txt)):
  all_clsa_pokok_perkara.append(CLSA(final_text[i]))

all_clsa_pokok_perkara[0:5]

['menghukum tergugat untuk membayar terlebih dahulu (uang muka) biaya transportasi dan akomodasi penggugat dalam proses pengajuan gugatan pemutusan hubungan kerja di pengadilan hubungan industrial pada pengadilan negeri ternate sebesar rp9.000.000,00 x 12 persidangan = rp108.000.000,00 (seratus delapan juta rupiah). setelah putusan ini dibacakan dalam persidangan;. menyatakan pemutusan hubungan kerja yang dilakukan tergugat terhadap penggugat pada tanggal 24 desember 2018 batal demi hukum;. menyatakan hubungan kerja antara penggugat dan tergugat putus karena pemutusan hubungan kerja yang dilakukan tergugat terhitung sejak putusan ini mempunyai kekuatan hukum tetap;. menghukum tergugat untuk membayar pesangon dan kompensasi sebesar 3 x pkb yakni 3 x 13,42 bulan x rp64.464.000,00 (enam puluh empat juta empat ratus enam puluh empat ribu rupiah) = rp2.595.320.640,00 (dua miliar lima ratus sembilan puluh lima juta tiga ratus dua puluh ribu enam ratus empat puluh rupiah);. menghukum tergugat

In [24]:
# fungsi load data txt
def load_data_to_list_lower(path_doc):
  path_doc = open(path, "r")
  documents_list = []
  for line in path_doc.readlines():
    lower = line.lower()
    text = lower.strip()
    documents_list.append(text)
  return documents_list

In [25]:
try:
  from rouge_score import rouge_scorer
except:
  !pip install rouge-score
  from rouge_score import rouge_scorer

In [26]:
pokok_perkara = '/content/drive/MyDrive/PROJECT/down_total_data/rangkuman_pakar/perbagian/3. pokok_perkara'
pokok_perkara = os.listdir(pokok_perkara)
pokok_perkara.sort()

hasil_pokok_perkara = []
for i in range(0,len(pokok_perkara)):
  path = "/content/drive/MyDrive/PROJECT/down_total_data/rangkuman_pakar/perbagian/3. pokok_perkara/"+str(pokok_perkara[i])
  # load dokumen
  list_pokok_perkara = load_data_to_list_lower(path)
  list_pokok_perkara =  "".join(list_pokok_perkara)

  hasil_pokok_perkara.append(list_pokok_perkara)
hasil_pokok_perkara[0:5]

['termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri ternate atas adanya pemutusan hubungan kerja (phk) oleh pemohon kasasi/tergugat. termohon kasasi/penggugat merasa pemutusan hubungan kerja (phk) yang dilakukan oleh pemohon kasasi/tergugat bertentangan dengan undang-undang nomor 13 tahun 2003 tentang ketenagakerjaan. oleh karena itu, pada intinya gugatan diajukan agar pemohon kasasi/tergugat membayar pesangon dan kompensasi sebesar 3 x pkb yakni 3 x 13,42 bulan x rp64.464.000,00 (enam puluh empat juta empat ratus enam puluh empat ribu rupiah) = rp2.595.320.640,00 (dua miliar lima ratus sembilan puluh lima juta tiga ratus dua puluh ribu enam ratus empat puluh rupiah). selain itu juga membayar upah dan tunjangan penggugat selama proses penyelesaian yakni tanggal 15 januari 2019 sampai ada putusan pengadilan yang berkekuatan hukum tetap sebesar rp74.800.266,00 (tujuh puluh empat juta delapan ratus ribu dua ratus enam puluh enam rupiah) 

In [27]:
def min_max_avg_eval(narasi_pd):
  narasi_tranpose = narasi_pd
  mean_p = narasi_tranpose['Preccsion'].astype(float)
  mean_p = mean_p.mean()*100
  mean_r = narasi_tranpose['Recall'].astype(float)
  mean_r = mean_r.mean()*100
  mean_f = narasi_tranpose['Rouge_Score'].astype(float)
  mean_f = mean_f.mean()*100

  max_p = narasi_tranpose['Preccsion'].astype(float)
  max_p = max_p.max()*100
  max_r = narasi_tranpose['Recall'].astype(float)
  max_r = max_r.max()*100
  max_f = narasi_tranpose['Rouge_Score'].astype(float)
  max_f = max_f.max()*100

  min_p = narasi_tranpose['Preccsion'].astype(float)
  min_p = min_p.min()*100
  min_r = narasi_tranpose['Recall'].astype(float)
  min_r = min_r.min()*100
  min_f = narasi_tranpose['Rouge_Score'].astype(float)
  min_f = min_f.min()*100

  print("Rata rata  : ")
  print("ROUGE Precision  : {:.3f} %".format(mean_p))
  print("ROUGE Recall     : {:.3f} %".format(mean_r))
  print("\nNilai Max: ")
  print("ROUGE Precision  : {:.3f} %".format(max_p))
  print("ROUGE Recall     : {:.3f} %".format(max_r))
  print("\nNilai Min: ")
  print("ROUGE Precision  : {:.3f} %".format(min_p))
  print("ROUGE Recall     : {:.3f} %".format(min_r))

In [28]:
def eval_rouge_1(narasi_pd):
  narasi_tranpose = narasi_pd
  mean_f = narasi_tranpose['Rouge_Score'].astype(float)
  mean_f = mean_f.mean()*100
  max_f = narasi_tranpose['Rouge_Score'].astype(float)
  max_f = max_f.max()*100
  min_f = narasi_tranpose['Rouge_Score'].astype(float)
  min_f = min_f.min()*100

  print("Rouge-1 Score")
  print("Rata rata       : {:.3f} %".format(mean_f))
  print("Nilai Max       : {:.3f} %".format(max_f))
  print("Nilai Min       : {:.3f} %".format(min_f))

In [29]:
def uji_rouge_1(pakar_nomor,rangkuman_sistem_list,hasil_pakar_list):
  scor_rouge = {}
  for i in range(0,len(pakar_nomor)):
    # deklarasi rangkuman
    sistem = rangkuman_sistem_list[i]
    pakar = hasil_pakar_list[i]
    pakar = pakar.lower()

    # uji evaluasi rogue
    scorerr = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scorerr = scorerr.score(sistem,pakar)

    # simpan nilai ke dalam list
    a = scorerr['rouge1']
    p = "{:.3f}".format(a[0])
    r = "{:.3f}".format(a[1])
    f = "{:.3f}".format(a[2])

    # simpan nilai ke dalam dictionary
    scor = [sistem,pakar,p,r,f]

    # # pelabelan dokumen
    nama = "nomor_putusan_{}".format(pakar_nomor[i])
    scor_rouge[nama] = scor
  return scor_rouge

In [30]:
def plotting_evaluasi(dataframe_data,judul):
  pp = dataframe_data
  p = []
  for i in pp['Preccsion']:
    i = float(i)
    a = i*100
    p.append(a)

  r = []
  for j in pp['Recall']:
    j = float(j)
    b = j*100
    r.append(b) 

  data = {'number': range(0,50),
          'precission': p,
          'recall': r,
        }
    
  df = pd.DataFrame(data,columns=['number','precission','recall'])
  df.plot(x ='number', y=['precission','recall'] , kind = 'line', figsize=(20,10),fontsize=20)
  plt.xlabel('Dokumen', size=30)
  plt.ylabel('Nilai', size=30)
  plt.title("Hasil Evaluasi {}".format(judul),fontsize=20)
  plt.show()

In [31]:
def plotting_evaluasi_rouge(dataframe_data,judul):
  pp = dataframe_data
  r_score_lsa = []
  for i in pp['r_score_lsa']:
    r_score_lsa.append(float(i))

  r_score_clsa = []
  for j in pp['r_score_clsa']:
    r_score_clsa.append(float(j)) 

  data = {'number': range(0,50),
          'r_score_lsa': r_score_lsa,
          'r_score_clsa': r_score_clsa,
        }
    
  df = pd.DataFrame(data,columns=['number','r_score_lsa','r_score_clsa'])
  df.plot(x ='number', y=['r_score_lsa','r_score_clsa'] , kind = 'line', figsize=(20,10),fontsize=20)
  plt.xlabel('Dokumen', size=30)
  plt.ylabel('Nilai', size=30)
  plt.title("Hasil Evaluasi {}".format(judul),fontsize=20)
  plt.show()

In [32]:
scor_pokok_perkara_lsa = uji_rouge_1(pokok_perkara,all_lsa_pokok_perkara,hasil_pokok_perkara)

# input dan visualisasi ke dataframe
pokok_perkara_lsa_pd = pd.DataFrame(scor_pokok_perkara_lsa)
pokok_perkara_lsa_pd.rename(index={0:'Pokok Perkara',1:'Pokok Perkara Summaries',2:'Preccsion',3:'Recall',4:'F1-score'}, inplace=True)
pokok_perkara_lsa_pd = pokok_perkara_lsa_pd.T
pokok_perkara_lsa_pd.to_csv (r'/content/drive/MyDrive/PROJECT/rouge_score_LSA_40.csv', index = False, header=True)
pokok_perkara_lsa_pd[0:5]

,Pokok Perkara,Pokok Perkara Summaries,Preccsion,Recall,F1-score
nomor_putusan_003.txt,menghukum tergugat untuk membayar terlebih dahulu (uang muka) biaya transportasi dan akomodasi penggugat dalam proses pengajuan gugatan pemutusan hubungan kerja di pengadilan hubungan industrial p...,termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri ternate atas adanya pemutusan hubungan kerja (phk) oleh pemohon kasasi/tergugat. termohon kasa...,0.793,0.582,0.671
nomor_putusan_006.txt,"menimbang, bahwa terhadap gugatan tersebut pengadilan hubungan industrial pada pengadilan negeri palembang telah memberikan putusan nomor 143/pdt.sus-phi/2019/pn plg. tanggal 23 maret 2020 yang am...",termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri palembang atas adanya pemutusan hubungan kerja (phk) oleh pemohon kasasi/tergugat. termohon ka...,0.329,0.511,0.400
nomor_putusan_007.txt,"menyatakan untuk segera menjatuhkan putusan sela, dan memutuskan terbih dahulu tuntutan dalam provisi ini yang ada dalam gugatan penggugat, dengan memerintahkan/menghukum tergugat membayarkan seka...",pemohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri bandung atas adanya perselisihan hak. namun tergugat tidak pernah hadir dalam persidangan sehin...,0.556,0.048,0.089
nomor_putusan_008.txt,menyatakan demi hukum tindakan tergugat yang tidak melaksanakan putusan/anjuran kepala dinas ketenagakerjaan pemerintah kota medan sesuai dengan surat nomor 567/1777/dkkm//2017 perihal anjuran tan...,termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri medan pada intinya antara lain: menyatakan dan menetapkan hubungan kerja antara penggugat deng...,0.685,0.488,0.570
nomor_putusan_009.txt,"menghukum tergugat secara sah dan meyakinkan untuk membayar hak- hak penggugat sebagaimana ketentuan pasal 96 ayat (1), (2), (3) dan (4) undang undang nomor 2 tahun 2004 tentang penyelesaian perse...",pemohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri jakarta pusat yang pada intinya memohon kepada pengadilan untuk memberikan putusan antara lain:...,0.626,0.386,0.478


In [33]:
scor_pokok_perkara_clsa = uji_rouge_1(pokok_perkara,all_clsa_pokok_perkara,hasil_pokok_perkara)

# input dan visualisasi ke dataframe
pokok_perkara_clsa_pd = pd.DataFrame(scor_pokok_perkara_clsa)
pokok_perkara_clsa_pd.rename(index={0:'Pokok Perkara',1:'Pokok Perkara Summaries',2:'Preccsion',3:'Recall',4:'F1-score'}, inplace=True)
pokok_perkara_clsa_pd = pokok_perkara_clsa_pd.T
pokok_perkara_clsa_pd.to_csv (r'/content/drive/MyDrive/PROJECT/rouge_score_CLSA_40.csv', index = False, header=True)
pokok_perkara_clsa_pd[0:5]

,Pokok Perkara,Pokok Perkara Summaries,Preccsion,Recall,F1-score
nomor_putusan_003.txt,menghukum tergugat untuk membayar terlebih dahulu (uang muka) biaya transportasi dan akomodasi penggugat dalam proses pengajuan gugatan pemutusan hubungan kerja di pengadilan hubungan industrial p...,termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri ternate atas adanya pemutusan hubungan kerja (phk) oleh pemohon kasasi/tergugat. termohon kasa...,0.883,0.396,0.547
nomor_putusan_006.txt,hubungan industrial pada pengadilan negeri palembang dan memohon kepada pengadilan untuk memberikan putusan sebagai berikut:. menyatakan hubungan kerja penggugat adalah demi hukum beralih menjadi ...,termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri palembang atas adanya pemutusan hubungan kerja (phk) oleh pemohon kasasi/tergugat. termohon ka...,0.657,0.421,0.513
nomor_putusan_007.txt,"menyatakan untuk segera menjatuhkan putusan sela, dan memutuskan terbih dahulu tuntutan dalam provisi ini yang ada dalam gugatan penggugat, dengan memerintahkan/menghukum tergugat membayarkan seka...",pemohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri bandung atas adanya perselisihan hak. namun tergugat tidak pernah hadir dalam persidangan sehin...,0.770,0.030,0.057
nomor_putusan_008.txt,hubungan industrial pada pengadilan negeri medan dan memohon kepada. menyatakan putusnya hubungan kerja antara penggugat dengan tergugat adalah karena putusan pengadilan (vide pasal 61 ayat (1) hu...,termohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri medan pada intinya antara lain: menyatakan dan menetapkan hubungan kerja antara penggugat deng...,0.873,0.443,0.588
nomor_putusan_009.txt,"hubungan industrial pada pengadilan negeri jakarta pusat dan memohon. menghukum tergugat secara sah dan meyakinkan untuk membayar hak- hak penggugat sebagaimana ketentuan pasal 96 ayat (1), (2), (...",pemohon kasasi/penggugat mengajukan gugatan di pengadilan hubungan industrial pada pengadilan negeri jakarta pusat yang pada intinya memohon kepada pengadilan untuk memberikan putusan antara lain:...,0.842,0.339,0.483
